In [33]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) #26 -? OUtput? 3
        self.bn1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8, 16, 3) #24,,5
        self.bn2 = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(16, 16, 3) #22,,5
        self.bn3 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)#11,,10

        self.conv4 = nn.Conv2d(16, 24, 3)#9,,12
        self.bn4 = nn.BatchNorm2d(24)
                
        self.conv5 = nn.Conv2d(24, 24, 3)#7,,12
        self.bn5 = nn.BatchNorm2d(24)
        
        self.conv6 = nn.Conv2d(24, 32, 3)#5,,12
        self.bn6 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(5)#1,,28
        
        self.fc1 = nn.Linear(32,10)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        
        x = self.bn2(F.relu(self.conv2(x)))
        
        x = self.pool1(self.bn3(F.relu(self.conv3(x))))
        x = self.dropout(x)

        x = self.bn4(F.relu(self.conv4(x)))
        x = self.dropout(x)
        
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.dropout(x)
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.pool2(x)
        
        x = x.view(-1,32)

        x = self.fc1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [35]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 16, 22, 22]           2,320
       BatchNorm2d-6           [-1, 16, 22, 22]              32
         MaxPool2d-7           [-1, 16, 11, 11]               0
           Dropout-8           [-1, 16, 11, 11]               0
            Conv2d-9             [-1, 24, 9, 9]           3,480
      BatchNorm2d-10             [-1, 24, 9, 9]              48
          Dropout-11             [-1, 24, 9, 9]               0
           Conv2d-12             [-1, 24, 7, 7]           5,208
      BatchNorm2d-13             [-1, 24, 7, 7]              48
          Dropout-14             [-1, 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [36]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [37]:
#every epoch tracking
from tqdm import tqdm
def train(model, device, train_loader, optimizer):
    #pbar = tqdm(train_loader)
    for epoch in range(19):
      model.train()
      correct = 0
      loss = 0
    
      for data, target in train_loader:
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          optimizer.step()
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

      print('\nTrain set: Accuracy: {}/{} ({:.4f}%)'.format(
        correct, len(train_loader.dataset),
          100. * correct / len(train_loader.dataset)))

      model.eval()
      test_loss = 0
      test_correct = 0
      with torch.no_grad():
          for data, target in test_loader:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
              pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
              test_correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)

      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
          test_loss, test_correct, len(test_loader.dataset),
          100. * test_correct / len(test_loader.dataset)))
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [38]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train(model, device, train_loader, optimizer)
test(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Train set: Accuracy: 54863/60000 (91.4383%)

Test set: Average loss: 0.0605, Accuracy: 9834/10000 (98.3400%)


Train set: Accuracy: 58955/60000 (98.2583%)

Test set: Average loss: 0.0371, Accuracy: 9904/10000 (99.0400%)


Test set: Average loss: 0.0371, Accuracy: 9904/10000 (99.0400%)

